In [208]:
# ============================================================

"""
This block prepares the environment and parsing tools used throughout the BLE dataset analysis.
It imports the required Python libraries for file handling, data processing, and visualization,
sets paths to the MQTT CSV logs, and defines the BLE gateway MAC address. It also pre-compiles
regular expressions for extracting MAC addresses, 128-bit UUIDs, and hexadecimal fields from
the BLE logs. Finally, it configures a logging system to produce structured, timestamped
diagnostic messages.
"""

# ============================================================

from __future__ import annotations
import os
import json
import logging                 # structured log messages
import re                      # regex for MAC / UUID extraction
from collections import Counter
from typing import Any, List, Tuple, Optional, Sequence
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# -----------------------------
# Data paths
# -----------------------------

# Root folder containing MQTT CSV logs
DATA_ROOT = "/home/nouman/nextome_data_CSV"

# CSV file to analyze
DEFAULT_CSV = os.path.join(DATA_ROOT, "Nextome_Monday_mqtt_beeta_Data_20260119_104602.csv")

# MAC address of the BLE gateway/controller device
GATEWAY_MAC = "0C:9A:42:18:A5:A4"


# -----------------
# Regex definitions
# -----------------

# Generic MAC address pattern (AA:BB:CC:DD:EE:FF)
# Matches any MAC address inside BLE log lines.
MAC_RE = re.compile(r"([0-9A-Fa-f]{2}(?::[0-9A-Fa-f]{2}){5})")

# Matches MAC addresses specifically in bluetoothctl-style lines:
# DEVICE_MAC_RE = re.compile(
#     r"Device\s+([0-9A-Fa-f]{2}(?::[0-9A-Fa-f]{2}){5})"
# )

# 128-bit UUID format used in BLE, 8-4-4-4-12 hex
UUID128_RE = re.compile(
    r"([0-9A-Fa-f]{8}-[0-9A-Fa-f]{4}-[0-9A-Fa-f]{4}-[0-9A-Fa-f]{4}-[0-9A-Fa-f]{12})"
)

# Single hexadecimal byte (00–FF)
# Used to validate TX Power, flags, and other 1-byte fields.
HEX_BYTE_RE = re.compile(r"^[0-9A-Fa-f]{2}$")


# -----------------------
# Logging configuration
# -----------------------

# INFO-level logging with timestamps for consistent diagnostic output.
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s] [%(levelname)s] %(message)s",
)


In [209]:
# ============================================================

"""
This block loads the MQTT CSV file and converts the nested JSON payloads into a structured form
suitable for BLE analysis. The `load_csv()` function reads the file and reports how many MQTT 
events (rows) were recorded. The `safe_json_load()` function safely parses each payload string 
into a dictionary, ensuring that malformed JSON does not interrupt execution. The `parse_payload()` 
function applies this parser to every row and extracts two key fields: the internal event timestamp
(`log_timestamp`) and the raw bluetoothctl scan output (`revelations_raw`), which contains MAC addresses,
RSSI values, and other BLE data. 
"""

# ============================================================


def load_csv(file_path: str) -> pd.DataFrame:
    
    logging.info(f"Loading CSV file: {file_path}")
    df = pd.read_csv(file_path)
    logging.info(f"Loaded {len(df)} rows and columns: {df.columns.tolist()}")
    return df


def safe_json_load(text: Any) -> dict:
    if not isinstance(text, str):
        return {}
    try:
        return json.loads(text)
    except Exception:
        logging.debug(f"Failed to parse JSON: {text}")
        return {}


def parse_payload(df: pd.DataFrame) -> pd.DataFrame:
    if "payload" not in df.columns:
        raise KeyError("CSV does not contain required 'payload' column.")

    # Parse JSON payloads
    df["payload_json"] = df["payload"].apply(safe_json_load)

    # Extract relevant fields from JSON payload
    df["log_timestamp"] = df["payload_json"].apply(lambda x: x.get("timestamp"))
    df["revelations_raw"] = df["payload_json"].apply(lambda x: x.get("revelations"))

    logging.info("Parsed JSON payloads: extracted 'log_timestamp' and 'revelations_raw'.")
    logging.info(f"Unique payload timestamps: {df['log_timestamp'].nunique()}")

    return df


# Load CSV
df = load_csv(DEFAULT_CSV)

# Parse JSON payload inside it
df = parse_payload(df)

# Show the first few entries of timestamps (for validation)
print("\n--- SAMPLE TIMESTAMPS FROM PAYLOAD JSON ---")
print(df["log_timestamp"].head().to_string(index=False))



[2026-01-29 17:47:10,909] [INFO] Loading CSV file: /home/nouman/nextome_data_CSV/Nextome_Monday_mqtt_beeta_Data_20260119_104602.csv
[2026-01-29 17:47:10,928] [INFO] Loaded 2362 rows and columns: ['timestamp_client', 'topic', 'payload']
[2026-01-29 17:47:10,939] [INFO] Parsed JSON payloads: extracted 'log_timestamp' and 'revelations_raw'.
[2026-01-29 17:47:10,940] [INFO] Unique payload timestamps: 2362



--- SAMPLE TIMESTAMPS FROM PAYLOAD JSON ---
2026-01-19_10:45:16.txt
2026-01-19_10:45:17.txt
2026-01-19_10:45:20.txt
2026-01-19_10:45:22.txt
2026-01-19_10:45:24.txt


In [210]:
# ============================================================

"""
This block inspects the raw MQTT metadata before further processing. It examines the `timestamp_client`
column to understand how MQTT message timing is recorded, checks the `topic` column to identify which 
MQTT topics are used, and inspects the `payload` column to view the raw JSON strings containing the BLE
scan data. This step clarifies the structure and content of the incoming MQTT messages and ensures that
the timestamp, topic, and payload formats are understood before parsing them in later stages.
"""

# ============================================================


# -------------------------------
# 1. Inspect timestamp_client
# -------------------------------
print(" TIMESTAMP_CLIENT (from MQTT client)")

ts_utc = pd.to_datetime(df["timestamp_client"], utc=True, errors="coerce")
ts_rome = ts_utc.dt.tz_convert("Europe/Rome")

print("Type (UTC parsed):", ts_utc.dtype)
print("Unique values:", ts_utc.nunique(dropna=True))

print("\nSample values (Rome local time):")
print(ts_rome.head(10).to_string(index=False))

# -------------------------------
# 2. Inspect topic column
# -------------------------------
print("\n\n TOPIC Column (MQTT topic names)")
print("Unique topics:", df["topic"].nunique())

print("\nList of all distinct topics:")
for t in df["topic"].unique():
    print(" -", t)

print("\nSample topic values:")
print(df["topic"].head(3).to_string(index=False))


# -------------------------------
# 3. Inspect payload column (raw JSON)
# -------------------------------
print("\n\n PAYLOAD Column (raw JSON strings)")
print("Showing first 5 payload entries:\n")

for i in range(5):
    print(f"[PAYLOAD {i}]")
    print(df["payload"].iloc[i])
    print("-" * 60)

 TIMESTAMP_CLIENT (from MQTT client)
Type (UTC parsed): datetime64[ns, UTC]
Unique values: 2362

Sample values (Rome local time):
2026-01-19 10:46:03.773435+01:00
2026-01-19 10:46:05.860730+01:00
2026-01-19 10:46:07.908109+01:00
2026-01-19 10:46:09.842031+01:00
2026-01-19 10:46:11.844614+01:00
2026-01-19 10:46:13.855791+01:00
2026-01-19 10:46:15.894994+01:00
2026-01-19 10:46:17.942692+01:00
2026-01-19 10:46:19.992564+01:00
2026-01-19 10:46:21.981552+01:00


 TOPIC Column (MQTT topic names)
Unique topics: 1

List of all distinct topics:
 - beeta

Sample topic values:
beeta
beeta
beeta


 PAYLOAD Column (raw JSON strings)
Showing first 5 payload entries:

[PAYLOAD 0]
{"timestamp": "2026-01-19_10:45:16.txt", "revelations": "[\u0001\u001b\u0002CHG\u0001\u001b\u0002] Device A8:42:E3:90:97:66 TxPower is nil\n[\u0001\u001b\u0002CHG\u0001\u001b\u0002] Device A8:42:E3:90:97:66 RSSI is nil\n[\u0001\u001b\u0002CHG\u0001\u001b\u0002] Device 7B:46:12:F3:5C:9F RSSI is nil\n[\u0001\u001b\u0002CHG\u00

In [211]:
# ============================================================
#   Remove ANSI Codes
# Purpose:
#   - bluetoothctl adds colors and control characters (ANSI escape codes)
#   - These make the text hard to parse with regex
#   - This block:
#       • Creates a new column: `revelations_clean`
#       • Shows a few examples: RAW vs CLEAN
#   This prepares the data for:
#       - MAC extraction
#       - UUID / iBeacon parsing
#       - Event analysis
# ============================================================

def clean_ansi(text: Any) -> Any:
    if not isinstance(text, str):
        return text

    # Remove ANSI escape sequences (e.g. \x1b[0;93m)
    text = re.sub(r"\x1B\[[0-?]*[ -/]*[@-~]", "", text)

    # Remove some additional control characters seen in logs
    text = text.replace("\x01", "").replace("\x02", "")

    return text

# Apply cleaning to revelations_raw
df["revelations_clean"] = df["revelations_raw"].apply(clean_ansi)

logging.info("Created 'revelations_clean' by removing ANSI/control codes.")

# Show a comparison: RAW vs CLEAN for a few rows
print("\n===================================================")
print("   SAMPLE REVELATIONS: RAW vs CLEAN")
print("===================================================\n")

for i in range(3):
    print(f"--- ROW {i} ---")
#     print("RAW:")
#     print(df["revelations_raw"].iloc[i])
    print("\nCLEAN:")
    print(df["revelations_clean"].iloc[i])
    print("-" * 60)

[2026-01-29 17:47:11,434] [INFO] Created 'revelations_clean' by removing ANSI/control codes.



   SAMPLE REVELATIONS: RAW vs CLEAN

--- ROW 0 ---

CLEAN:
[CHG] Device A8:42:E3:90:97:66 TxPower is nil
[CHG] Device A8:42:E3:90:97:66 RSSI is nil
[CHG] Device 7B:46:12:F3:5C:9F RSSI is nil
[CHG] Device BC:57:29:03:7B:37 RSSI is nil
[CHG] Device 29:08:03:CA:1D:1E RSSI is nil
[CHG] Device BC:57:29:02:89:2B RSSI is nil
[CHG] Device BC:57:29:02:89:27 RSSI is nil
[CHG] Device 18:EF:3A:6E:2D:56 RSSI is nil
[CHG] Controller 0C:9A:42:18:A5:A4 Discovering: no
Discovery started
[CHG] Controller 0C:9A:42:18:A5:A4 Discovering: yes
[CHG] Device A8:42:E3:90:97:66 RSSI: -72
[CHG] Device 59:84:66:5D:D9:38 RSSI: -88
[CHG] Device 59:84:66:5D:D9:38 ManufacturerData Key: 0x0075
[CHG] Device 59:84:66:5D:D9:38 ManufacturerData Value:
  d6 ca 67 19 87 df                                ..g...          

------------------------------------------------------------
--- ROW 1 ---

CLEAN:
[CHG] Device 59:84:66:5D:D9:38 RSSI is nil
[CHG] Device A8:42:E3:90:97:66 RSSI is nil
[CHG] Controller 0C:9A:42:18:A5:A4 Di

In [212]:
# ============================================================

"""
This block analyzes controller-level activity in the Bluetooth logs by scanning the `revelations_clean`
column for patterns related to the BLE controller—such as `[CHG] Controller`, `Discovering: yes/no`, 
`Powered: yes/no`, and `Pairable: yes/no`. It counts how often each controller state occurs and determines 
how many MQTT messages contain any controller-related event. This helps describe the behaviour of the BLE 
scanner itself so that later analysis can clearly separate controller state changes from actual device-level 
events.
"""

# ============================================================

controller_patterns = {
    "[CHG] Controller": 0,
    "[NEW] Controller": 0,
    "[DEL] Controller": 0,
    "Discovering: yes": 0,
    "Discovering: no": 0,
    "Powered: yes": 0,
    "Powered: no": 0,
    "Pairable: yes": 0,
    "Pairable: no": 0,
}

total_rows = len(df)

for text in df["revelations_clean"]: #text = one block of text for one timestamp.
    if not isinstance(text, str):
        continue

    for pattern in controller_patterns.keys():
        if pattern in text:
            controller_patterns[pattern] += 1

# Count how many rows contain ANY controller keyword
def has_any_controller_event(text: str) -> bool:
    if not isinstance(text, str):
        return False
    if "Controller " in text:
        return True
    # also consider lines that only say "Discovering: ..." or "Powered: ..."
    for key in ["Discovering:", "Powered:", "Pairable:"]:
        if key in text:
            return True
    return False

rows_with_controller = sum(
    1 for txt in df["revelations_clean"] if has_any_controller_event(txt)
)

print("\n===================================================")
print("   CONTROLLER-LEVEL EVENTS ")
print("===================================================\n")

print(f"Total MQTT messages (rows): {total_rows}")
# print(f"Rows containing at least one controller-related event: {rows_with_controller}")
print()

print("Event counts (controller-level):")
for pattern, count in controller_patterns.items():
    print(f"  {pattern:<18} --> {count} occurrences")


   CONTROLLER-LEVEL EVENTS 

Total MQTT messages (rows): 2362

Event counts (controller-level):
  [CHG] Controller   --> 2361 occurrences
  [NEW] Controller   --> 0 occurrences
  [DEL] Controller   --> 30 occurrences
  Discovering: yes   --> 2314 occurrences
  Discovering: no    --> 2043 occurrences
  Powered: yes       --> 1 occurrences
  Powered: no        --> 30 occurrences
  Pairable: yes      --> 0 occurrences
  Pairable: no       --> 0 occurrences


In [213]:
# ============================================================

"""
This block scans `revelations_clean` for `[NEW]`, `[CHG]`, and `[DEL]` device events, counts their
occurrences, and identifies how many MQTT messages contain device activity. `[NEW]` marks first 
detection, `[CHG]` updates an existing device, and `[DEL]` indicates removal or timeout. These events 
are later used to compute device entry/exit times and presence duration.
"""

# ============================================================

device_new_count = 0
device_chg_count = 0
device_del_count = 0

rows_with_device_events = 0

for text in df["revelations_clean"]:
    if not isinstance(text, str):
        continue

    has_device = False

    for line in text.splitlines():
        line_stripped = line.strip()
        if line_stripped.startswith("[NEW] Device "):
            device_new_count += 1
            has_device = True
        elif line_stripped.startswith("[CHG] Device "):
            device_chg_count += 1
            has_device = True
        elif line_stripped.startswith("[DEL] Device "):
            device_del_count += 1
            has_device = True

    if has_device:
        rows_with_device_events += 1

total_device_events = device_new_count + device_chg_count + device_del_count

print("\n===================================================")
print("   DEVICE-LEVEL EVENTS  ")
print("===================================================\n")

print(f"Rows containing at least one DEVICE event: {rows_with_device_events}")
print(f"Total DEVICE events (NEW + CHG + DEL):    {total_device_events}\n")

print("DEVICE-LEVEL EVENTS  type:")
print(f"  [NEW] Device  --> {device_new_count} events")
print(f"  [CHG] Device  --> {device_chg_count} events")
print(f"  [DEL] Device  --> {device_del_count} events")


   DEVICE-LEVEL EVENTS  

Rows containing at least one DEVICE event: 2323
Total DEVICE events (NEW + CHG + DEL):    23825

DEVICE-LEVEL EVENTS  type:
  [NEW] Device  --> 2815 events
  [CHG] Device  --> 18363 events
  [DEL] Device  --> 2647 events


In [214]:
# ============================================================

"""
This block extracts all MAC addresses from the cleaned Bluetooth logs, flattens them into a single list, 
and counts how many times each MAC appears. It separates the gateway MAC from real BLE devices and produces 
a summary of how many unique devices were detected and how frequently each one appeared in the logs. 
"""

# ============================================================
#
# -----------------------------
# 1. Extract MAC list per row
# -----------------------------
def extract_mac_list(block_text: Any) -> list[str]:
    """
    Extract ALL MAC addresses appearing in a bluetoothctl block.
    
    This includes:
      - Device MACs (BLE devices)
      - Controller MAC (gateway)
    
    Filtering of gateway MAC will happen later.
    """
    if not isinstance(block_text, str):
        return []
    return [m.upper() for m in MAC_RE.findall(block_text)]


df["mac_list"] = df["revelations_clean"].apply(extract_mac_list)


# -----------------------------
# 2. Flatten into a single list
# -----------------------------
all_macs = []
for macs in df["mac_list"]:
    if isinstance(macs, list):
        all_macs.extend(macs)

# -----------------------------
# 3. Unique devices
# -----------------------------
unique_macs = sorted(set(all_macs))
total_unique = len(unique_macs)


# -----------------------------
# 4. Event counts per MAC
# -----------------------------
from collections import Counter
mac_event_counts = Counter(all_macs)


# -----------------------------
# 5. Identify gateway MAC
# -----------------------------
GATEWAY_MAC = "0C:9A:42:18:A5:A4"

gateway_event_count = mac_event_counts.get(GATEWAY_MAC, 0)

# Devices only = remove gateway
device_macs = [m for m in unique_macs if m != GATEWAY_MAC]
total_device_count = len(device_macs)


# -----------------------------
# 6. PRINT SUMMARY
# -----------------------------
print("\n===================================================")
print("      BLE DEVICE IDENTIFICATION ")
print("===================================================\n")

print(f"Total MAC addresses detected : {total_unique}")
print(f"Total devices (excluding gateway): {total_device_count}")
print(f"Gateway MAC ({GATEWAY_MAC}) detected with: {gateway_event_count} events\n")

print("\n===================================================")
print("     LIST OF DEVICES (MAC → Event Count)")
print("===================================================\n")

for mac, cnt in mac_event_counts.items():
    print(f"{mac:<18} ---------> {cnt} events")

# print("\nTotal devices printed:", len(mac_event_counts))


      BLE DEVICE IDENTIFICATION 

Total MAC addresses detected : 311
Total devices (excluding gateway): 310
Gateway MAC (0C:9A:42:18:A5:A4) detected with: 4655 events


     LIST OF DEVICES (MAC → Event Count)

A8:42:E3:90:97:66  ---------> 1874 events
7B:46:12:F3:5C:9F  ---------> 25 events
BC:57:29:03:7B:37  ---------> 198 events
29:08:03:CA:1D:1E  ---------> 37 events
BC:57:29:02:89:2B  ---------> 450 events
BC:57:29:02:89:27  ---------> 690 events
18:EF:3A:6E:2D:56  ---------> 577 events
0C:9A:42:18:A5:A4  ---------> 4655 events
59:84:66:5D:D9:38  ---------> 9 events
BC:57:29:02:89:2E  ---------> 408 events
BC:57:29:02:89:2D  ---------> 406 events
54:98:43:90:50:F5  ---------> 38 events
5F:B6:9E:0C:DC:37  ---------> 14 events
6D:4C:BB:59:2F:45  ---------> 17 events
7F:11:01:A6:93:84  ---------> 9 events
1C:8B:73:1B:D1:00  ---------> 32 events
C1:08:15:05:54:4F  ---------> 93 events
DD:34:02:06:34:2E  ---------> 922 events
7F:17:4C:CA:2C:96  ---------> 48 events
45:8A:73:06:A8:AD  

In [215]:
# ============================================================
"""
Extract all device-level bluetoothctl events ([NEW]/[CHG]/[DEL]) WITH timestamps,
group them by MAC, and print full event history per device.

Output per MAC:
  - timestamp (from df["ts_utc"] if present, else parsed from df["timestamp_client"])
  - event_type (NEW/CHG/DEL)
  - raw event line
"""
# ============================================================

import pandas as pd

# Ensure a timestamp column exists (use ts_utc if already created; else create it)
if "ts_utc" not in df.columns:
    df = df.copy()
    df["ts_utc"] = pd.to_datetime(df["timestamp_client"], utc=True, errors="coerce")

mac_events_dict = {}  # MAC -> list of dicts {ts, event_type, line}

for ts, text in zip(df["ts_utc"].tolist(), df["revelations_clean"].tolist()):
    if not isinstance(text, str):
        continue

    for line in text.splitlines():
        line = line.strip()

        if line.startswith("[NEW] Device "):
            event_type = "NEW"
        elif line.startswith("[CHG] Device "):
            event_type = "CHG"
        elif line.startswith("[DEL] Device "):
            event_type = "DEL"
        else:
            continue

        m = MAC_RE.search(line)
        if not m:
            continue

        mac = m.group(1).upper()

        if mac not in mac_events_dict:
            mac_events_dict[mac] = []

        mac_events_dict[mac].append({
            "timestamp": ts,
            "event_type": event_type,
            "line": line,
        })

# Optional: sort events per MAC by timestamp
for mac in mac_events_dict:
    mac_events_dict[mac] = sorted(
        mac_events_dict[mac],
        key=lambda x: (pd.Timestamp.min if pd.isna(x["timestamp"]) else x["timestamp"])
    )

# PRINT RESULTS
print("\n===================================================")
print("   ALL DEVICE EVENTS GROUPED BY MAC (WITH TIMESTAMP)")
print("===================================================\n")

for mac, events in mac_events_dict.items():
    print(f"\n▶ MAC: {mac}   (Total events: {len(events)})")
    print("-" * 80)
    for e in events:
        ts_str = str(e["timestamp"]) if not pd.isna(e["timestamp"]) else "NaT"
        print(f"{ts_str} | {e['line']}")


   ALL DEVICE EVENTS GROUPED BY MAC (WITH TIMESTAMP)


▶ MAC: A8:42:E3:90:97:66   (Total events: 1874)
--------------------------------------------------------------------------------
2026-01-19 09:46:03.773435+00:00 | [CHG] Device A8:42:E3:90:97:66 TxPower is nil
2026-01-19 09:46:03.773435+00:00 | [CHG] Device A8:42:E3:90:97:66 RSSI is nil
2026-01-19 09:46:03.773435+00:00 | [CHG] Device A8:42:E3:90:97:66 RSSI: -72
2026-01-19 09:46:05.860730+00:00 | [CHG] Device A8:42:E3:90:97:66 RSSI is nil
2026-01-19 09:46:13.855791+00:00 | [CHG] Device A8:42:E3:90:97:66 RSSI: -70
2026-01-19 09:46:15.894994+00:00 | [CHG] Device A8:42:E3:90:97:66 RSSI is nil
2026-01-19 09:46:17.942692+00:00 | [CHG] Device A8:42:E3:90:97:66 RSSI: -76
2026-01-19 09:46:19.992564+00:00 | [CHG] Device A8:42:E3:90:97:66 RSSI is nil
2026-01-19 09:46:40.165135+00:00 | [CHG] Device A8:42:E3:90:97:66 RSSI: -76
2026-01-19 09:46:40.165135+00:00 | [CHG] Device A8:42:E3:90:97:66 TxPower: 9
2026-01-19 09:46:40.165135+00:00 | [CHG] 

In [216]:
# ============================================================
"""
Scan cleaned Bluetooth logs for iBeacon ManufacturerData and extract:
    (MAC, UUID, Major, Minor)

Behavior:
- Does NOT assume ManufacturerData Value starts at byte 0 with "02 15".
- Searches for iBeacon header inside the collected bytes:
    - 4C 00 02 15  (company ID included in value by some loggers)
    - FF 4C 00 02 15 (raw AD type included by some loggers)
    - 02 15        (company ID stripped; common in your logs)
- Does NOT require "ManufacturerData Key: 0x004c" (since your revelations_clean may drop it).

Output:
- mac_beacon_df: unique (mac, uuid, major, minor) tuples
- mac_beacon_agg: one representative tuple per MAC
- device counts excluding GATEWAY_MAC
"""
# ============================================================

from typing import Any, List, Tuple, Optional

# NOTE: assumes these already exist in your environment:
#   - df (pandas DataFrame)
#   - pd (pandas imported)
#   - MAC_RE (regex that captures MAC address in group(1))
#   - HEX_BYTE_RE (regex matching a single hex byte token, e.g. r"[0-9A-Fa-f]{2}")
#   - GATEWAY_MAC (gateway MAC string)

# -----------------------------
# Helpers
# -----------------------------

def _find_subseq(haystack: List[str], needle: List[str]) -> int:
    """Return first index where needle occurs in haystack (case-insensitive); else -1."""
    h = [x.lower() for x in haystack]
    n = [x.lower() for x in needle]
    ln = len(n)
    for i in range(0, len(h) - ln + 1):
        if h[i:i+ln] == n:
            return i
    return -1


def _find_ibeacon_uuid_start(hex_bytes: List[str]) -> Optional[int]:
    """
    Return index where the 16-byte UUID starts inside hex_bytes, or None.
    Supports:
      - 4C 00 02 15 <UUID...>
      - FF 4C 00 02 15 <UUID...>
      - 02 15 <UUID...>
    """
    idx = _find_subseq(hex_bytes, ["4c", "00", "02", "15"])
    if idx != -1:
        return idx + 4

    idx = _find_subseq(hex_bytes, ["ff", "4c", "00", "02", "15"])
    if idx != -1:
        return idx + 5

    idx = _find_subseq(hex_bytes, ["02", "15"])
    if idx != -1:
        return idx + 2

    return None


def _uuid_bytes_to_str(uuid_bytes: List[str]) -> str:
    """Convert 16 hex-byte strings into canonical UUID string."""
    return (
        "".join(uuid_bytes[0:4]) + "-" +
        "".join(uuid_bytes[4:6]) + "-" +
        "".join(uuid_bytes[6:8]) + "-" +
        "".join(uuid_bytes[8:10]) + "-" +
        "".join(uuid_bytes[10:16])
    ).lower()


# -----------------------------
# Parse iBeacon frames per block
# -----------------------------

def extract_ibeacon_tuples(block_text: Any) -> List[Tuple[str, str, int, int]]:
    """
    From one 'revelations_clean' block, extract iBeacon tuples:
        (mac, beacon_uuid, major_dec, minor_dec)

    Logic:
      - Track current MAC from lines like "[CHG] Device XX:YY:..."
      - When we hit "ManufacturerData Value:" under that MAC:
          - collect subsequent hex lines
          - search inside bytes for iBeacon header
          - parse UUID, major, minor (big-endian)
    """
    results: List[Tuple[str, str, int, int]] = []
    if not isinstance(block_text, str):
        return results

    lines = block_text.splitlines()
    current_mac: Optional[str] = None
    i = 0

    while i < len(lines):
        line = lines[i].strip()

        # Update current MAC from "Device" lines
        m = MAC_RE.search(line)
        if m:
            current_mac = m.group(1).upper()

        # ManufacturerData Value block
        if "ManufacturerData Value:" in line and current_mac is not None:
            hex_bytes: List[str] = []
            j = i + 1

            # Collect subsequent lines containing hex bytes
            while j < len(lines):
                t = lines[j].strip()
                if not t:
                    break

                tokens = t.split()
                hex_tokens = [h for h in tokens if HEX_BYTE_RE.fullmatch(h)]
                if not hex_tokens:
                    break

                hex_bytes.extend(hex_tokens)
                j += 1

            # Find iBeacon UUID start inside these bytes
            start = _find_ibeacon_uuid_start(hex_bytes)

            # Need UUID(16) + major(2) + minor(2) + tx(1)
            if start is not None and len(hex_bytes) >= start + 16 + 2 + 2 + 1:
                uuid_bytes  = hex_bytes[start : start + 16]
                major_bytes = hex_bytes[start + 16 : start + 18]
                minor_bytes = hex_bytes[start + 18 : start + 20]

                beacon_uuid = _uuid_bytes_to_str(uuid_bytes)
                major_dec = int("".join(major_bytes), 16)  # big-endian
                minor_dec = int("".join(minor_bytes), 16)  # big-endian

                results.append((current_mac, beacon_uuid, major_dec, minor_dec))

            # Skip past ManufacturerData block
            i = j
        else:
            i += 1

    return results


# -----------------------------
# Apply to all rows → flat table
# -----------------------------

df["mac_beacon_tuples"] = df["revelations_clean"].apply(extract_ibeacon_tuples)

flat_rows: List[Tuple[str, str, int, int]] = []
for tuples_ in df["mac_beacon_tuples"]:
    if isinstance(tuples_, list):
        flat_rows.extend(tuples_)

mac_beacon_df = (
    pd.DataFrame(flat_rows, columns=["mac", "beacon_uuid", "major", "minor"])
    .drop_duplicates()
    .reset_index(drop=True)
)

# One representative iBeacon tuple per MAC (first seen / lowest sort)
if not mac_beacon_df.empty:
    mac_beacon_agg = (
        mac_beacon_df
        .sort_values(["mac", "beacon_uuid", "major", "minor"])
        .groupby("mac", as_index=False)
        .first()
    )
else:
    mac_beacon_agg = pd.DataFrame(columns=["mac", "beacon_uuid", "major", "minor"])


print("\n===================================================")
print("    iBeacon Tuples Extracted")
print("===================================================\n")

print(f"Total iBeacon records : {len(mac_beacon_df)}")
print(f"Unique MACs with at least one iBeacon record:  {mac_beacon_df['mac'].nunique()}")
print(f"Unique beacon UUIDs:                           {mac_beacon_df['beacon_uuid'].nunique()}\n")

if not mac_beacon_df.empty:
    unique_uuid_values = mac_beacon_df["beacon_uuid"].dropna().unique()
    print("List of unique beacon UUIDs:")
    for u in unique_uuid_values:
        print("  →", u)
    print()

    print("Sample iBeacon tuples:")
    print(mac_beacon_df.head(25).to_string(index=False))


# -----------------------------
# Device counts (excluding gateway)
# -----------------------------

all_macs_from_logs: List[str] = []
for macs in df.get("mac_list", []):
    if isinstance(macs, list):
        all_macs_from_logs.extend([m.upper() for m in macs])

all_unique_macs = sorted(set(all_macs_from_logs))
all_unique_devices = [m for m in all_unique_macs if m != GATEWAY_MAC]

total_devices = len(all_unique_devices)
devices_with_tuple = mac_beacon_agg["mac"].nunique()
devices_without_tuple = total_devices - devices_with_tuple

print("===================================================\n")
print(f"Total devices (excluding gateway): {total_devices}")
print(f"Devices with (UUID, Major, Minor) tuple: {devices_with_tuple}")
print(f"Devices WITHOUT iBeacon tuple  {devices_without_tuple}\n")



    iBeacon Tuples Extracted

Total iBeacon records : 16
Unique MACs with at least one iBeacon record:  16
Unique beacon UUIDs:                           2

List of unique beacon UUIDs:
  → 7777772e-6b6b-6d63-6e2e-636f6d000001
  → acfd065e-c3c0-11e3-9bbe-1a514932ac01

Sample iBeacon tuples:
              mac                          beacon_uuid  major  minor
BC:57:29:03:7B:62 7777772e-6b6b-6d63-6e2e-636f6d000001      1      1
BC:57:29:03:7F:06 7777772e-6b6b-6d63-6e2e-636f6d000001      5  63627
DD:34:02:06:34:2E 7777772e-6b6b-6d63-6e2e-636f6d000001      2  54371
BC:57:29:03:7B:37 7777772e-6b6b-6d63-6e2e-636f6d000001      1      1
DD:34:02:06:FE:30 7777772e-6b6b-6d63-6e2e-636f6d000001      3  34701
BC:57:29:02:89:24 7777772e-6b6b-6d63-6e2e-636f6d000001      5  39126
BC:57:29:02:9E:D3 7777772e-6b6b-6d63-6e2e-636f6d000001      5  44677
BC:57:29:02:89:27 7777772e-6b6b-6d63-6e2e-636f6d000001      5  39129
BC:57:29:02:89:28 7777772e-6b6b-6d63-6e2e-636f6d000001      5  39130
DD:34:02:06:FE:3B

In [233]:
# ============================================================

"""
This block first summarizes the iBeacon parameter space by listing all unique UUID, Major, and Minor values
found in `mac_beacon_df`. It then categorizes devices into two groups: those that have an associated iBeacon
tuple `(UUID, Major, Minor)` (from `mac_beacon_agg`) and those that do not (devices seen in the logs but without
iBeacon identifiers). For each category, it prints the count and lists the corresponding MAC addresses, providing a clear view of which devices are identifiable via iBeacon and which are only observed at MAC level.
"""

# ============================================================


# ---- 1. Unique UUIDs ----
unique_uuid_values = sorted(mac_beacon_df["beacon_uuid"].dropna().unique())
print(f"Total unique UUIDs: {len(unique_uuid_values)}")
for u in unique_uuid_values:
    print("  →", u)

# ---- 2. Unique MAJOR ----
unique_major_values = sorted(mac_beacon_df["major"].dropna().unique())
print(f"Total unique MAJOR values: {len(unique_major_values)}")
for mj in unique_major_values:
    print("  →", mj)

# ---- 3. Unique MINOR ----
unique_minor_values = sorted(mac_beacon_df["minor"].dropna().unique())
print(f"Total unique MINOR values: {len(unique_minor_values)}")
for mn in unique_minor_values:
    print("  →", mn)


# ============================================================
#  DEVICE CATEGORIES 
# ============================================================


# ----  devices (no tuple) ----
anonymous_macs = sorted(set(all_unique_devices) - set(mac_beacon_agg["mac"]))
print(f"Devices WITHOUT iBeacon tuple : {len(anonymous_macs)}")
print("WITHOUT iBeacon tuple MACs:")
for m in anonymous_macs:
    print("  →", m)

# ---- Devices with iBeacon tuple ----
identified_devices = mac_beacon_agg.sort_values("mac")
print(f"Devices with (UUID, Major, Minor) tuple: {len(identified_devices)}")
print("Identifiable devices (MAC | UUID | Major | Minor):")
print(identified_devices.to_string(index=False))


Total unique UUIDs: 2
  → 7777772e-6b6b-6d63-6e2e-636f6d000001
  → acfd065e-c3c0-11e3-9bbe-1a514932ac01
Total unique MAJOR values: 4
  → 1
  → 2
  → 3
  → 5
Total unique MINOR values: 15
  → 1
  → 20328
  → 34701
  → 34712
  → 39121
  → 39126
  → 39129
  → 39130
  → 39133
  → 39135
  → 39136
  → 39137
  → 44677
  → 54371
  → 63627
Devices WITHOUT iBeacon tuple : 294
WITHOUT iBeacon tuple MACs:
  → 00:A5:54:20:52:2D
  → 01:03:16:3C:B7:48
  → 04:1D:36:02:01:D9
  → 04:42:62:82:5E:C4
  → 05:0E:31:20:CD:3A
  → 05:32:3E:38:EE:3D
  → 0B:B8:14:57:35:83
  → 0D:A6:98:83:B0:38
  → 0E:B7:28:BC:30:34
  → 0F:F2:9F:A3:AE:8B
  → 12:3C:21:D4:3C:FE
  → 12:94:81:A1:47:BB
  → 13:FE:60:F9:C5:F9
  → 14:9B:DC:8F:61:49
  → 15:AF:FF:E9:50:63
  → 16:83:E8:8F:63:D7
  → 16:EA:53:AB:A5:DD
  → 18:EF:3A:6E:2D:56
  → 19:51:1E:C4:5E:4D
  → 1B:73:D9:F4:E4:CD
  → 1B:B0:DA:6C:EB:CF
  → 1B:EC:C5:49:91:81
  → 1C:8B:73:1B:D1:00
  → 1D:14:64:84:62:F7
  → 1E:82:74:B9:84:D9
  → 1E:A6:45:86:B9:24
  → 1E:DB:EC:FA:12:E2
  → 20:8A

In [234]:
# =========================
# Beacon-only Layer 2 table
# (Session-based presence, scalar time output)
# =========================
# Output (MAC-only):
#   mac, first_seen, last_seen, num_sessions,
#   total_presence_seconds, total_absence_seconds
# Index starts from 1 (not 0)

import pandas as pd

# --- Config ---
GAP_SECONDS = 120
GAP = pd.Timedelta(seconds=GAP_SECONDS)

# --- Beacon MAC inventory ---
beacon_macs = set(mac_beacon_agg["mac"].astype(str).str.upper().unique())

# --- Parse timestamps ---
df = df.copy()
df["ts_utc"] = pd.to_datetime(df["timestamp_client"], utc=True, errors="coerce")

# --- Ensure mac_list exists ---
if "mac_list" not in df.columns:
    def _extract_mac_list(block):
        if not isinstance(block, str):
            return []
        return [m.upper() for m in MAC_RE.findall(block)]
    df["mac_list"] = df["revelations_clean"].apply(_extract_mac_list)

# --- Build beacon observations ---
obs_rows = []
for ts, macs in zip(df["ts_utc"], df["mac_list"]):
    if pd.isna(ts) or not isinstance(macs, list):
        continue
    for m in macs:
        mu = m.upper()
        if mu in beacon_macs:
            obs_rows.append((mu, ts))

obs = (
    pd.DataFrame(obs_rows, columns=["mac", "ts"])
    .drop_duplicates()
    .sort_values(["mac", "ts"])
    .reset_index(drop=True)
)

# --- Compute session-based metrics ---
metrics = []

for mac, g in obs.groupby("mac", sort=False):
    t = g["ts"].sort_values().reset_index(drop=True)

    first_seen = t.iloc[0]
    last_seen  = t.iloc[-1]

    gaps = t.diff()

    # session boundaries
    is_new_session = gaps.isna() | (gaps > GAP)
    session_ids = is_new_session.cumsum()

    session_starts = t.groupby(session_ids).first()
    session_ends   = t.groupby(session_ids).last()

    # session durations (strict observation window)
    session_durations = session_ends - session_starts

    total_presence = session_durations.sum()
    total_absence  = gaps[gaps > GAP].sum()

    metrics.append({
        "mac": mac,
        "first_seen": first_seen,
        "last_seen": last_seen,
        "num_sessions": len(session_durations),
        "total_presence_seconds": total_presence.total_seconds(),
        "total_absence_seconds": total_absence.total_seconds(),
    })

df_beacon_metrics = (
    pd.DataFrame(metrics)
    .sort_values(["first_seen", "mac"])
    .reset_index(drop=True)
)

# --- Human-friendly index ---
df_beacon_metrics.index = df_beacon_metrics.index + 1

# --- Final table ---
df_beacon_metrics


,mac,first_seen,last_seen,num_sessions,total_presence_seconds,total_absence_seconds
1,BC:57:29:02:89:27,2026-01-19 09:46:03.773435+00:00,2026-01-19 11:45:59.469857+00:00,7,5504.663459,1691.032963
2,BC:57:29:02:89:2B,2026-01-19 09:46:03.773435+00:00,2026-01-19 11:45:53.261634+00:00,6,5657.159073,1532.329126
3,BC:57:29:03:7B:37,2026-01-19 09:46:03.773435+00:00,2026-01-19 11:43:49.923053+00:00,21,2092.322986,4973.826632
4,BC:57:29:02:89:2E,2026-01-19 09:46:05.860730+00:00,2026-01-19 11:45:43.084721+00:00,8,5137.026785,2040.197206
5,BC:57:29:02:89:2D,2026-01-19 09:46:07.908109+00:00,2026-01-19 11:45:59.469857+00:00,10,4787.514331,2404.047417
6,DD:34:02:06:34:2E,2026-01-19 09:46:11.844614+00:00,2026-01-19 11:45:59.469857+00:00,6,5618.254257,1569.370986
7,BC:57:29:02:89:24,2026-01-19 09:46:15.894994+00:00,2026-01-19 11:45:59.469857+00:00,7,5383.544656,1800.030207
8,EB:B3:B3:8F:D6:C8,2026-01-19 09:46:19.992564+00:00,2026-01-19 11:45:53.261634+00:00,2,6244.689664,928.579406
9,BC:57:29:03:7F:06,2026-01-19 09:46:21.981552+00:00,2026-01-19 11:45:59.469857+00:00,12,3864.058358,3313.429947
10,BC:57:29:02:9E:D3,2026-01-19 09:46:24.001904+00:00,2026-01-19 11:45:53.261634+00:00,17,2741.733996,4427.525734


In [235]:
import pandas as pd

# obs: DataFrame with columns ["mac", "ts"]
# GAP: pd.Timedelta (e.g., pd.Timedelta(seconds=60))

# --- Safety: ensure ts is datetime ---
obs = obs.copy()
obs["ts"] = pd.to_datetime(obs["ts"], errors="coerce")
obs = obs.dropna(subset=["mac", "ts"])

session_rows = []

# --- Build per-session start/end for each MAC ---
for mac, g in obs.groupby("mac", sort=False):
    t = g["ts"].sort_values().reset_index(drop=True)
    if t.empty:
        continue

    gaps = t.diff()

    # New session when first row OR gap > GAP
    session_id = (gaps.isna() | (gaps > GAP)).cumsum()  # 1..N per MAC

    starts = t.groupby(session_id).first()
    ends   = t.groupby(session_id).last()

    for sid in starts.index:
        session_rows.append({
            "mac": mac,
            "session_id": int(sid),
            "session_start": starts.loc[sid],
            "session_end": ends.loc[sid],
        })

df_beacon_session_detail = (
    pd.DataFrame(session_rows)
      .sort_values(["mac", "session_id"])
      .reset_index(drop=True)
)

# --- Print per-MAC session list ---
for mac, g in df_beacon_session_detail.groupby("mac", sort=False):
    total_sessions = int(g["session_id"].max()) if not g.empty else 0
    print(f"\nMAC: {mac} | total_sessions: {total_sessions}")
    for _, r in g.iterrows():
        print(f"  Session {int(r['session_id'])}: {r['session_start']} | {r['session_end']}")



MAC: BC:57:29:02:89:1F | total_sessions: 5
  Session 1: 2026-01-19 09:47:48.977898+00:00 | 2026-01-19 09:59:39.936850+00:00
  Session 2: 2026-01-19 10:15:24.812389+00:00 | 2026-01-19 10:48:22.802454+00:00
  Session 3: 2026-01-19 10:51:35.419337+00:00 | 2026-01-19 11:16:48.401733+00:00
  Session 4: 2026-01-19 11:20:48.755066+00:00 | 2026-01-19 11:33:14.223540+00:00
  Session 5: 2026-01-19 11:35:52.330358+00:00 | 2026-01-19 11:45:43.084721+00:00

MAC: BC:57:29:02:89:24 | total_sessions: 7
  Session 1: 2026-01-19 09:46:15.894994+00:00 | 2026-01-19 09:59:52.239967+00:00
  Session 2: 2026-01-19 10:16:51.305294+00:00 | 2026-01-19 10:41:54.703448+00:00
  Session 3: 2026-01-19 10:44:04.953941+00:00 | 2026-01-19 10:57:55.635746+00:00
  Session 4: 2026-01-19 11:00:38.557446+00:00 | 2026-01-19 11:28:28.346955+00:00
  Session 5: 2026-01-19 11:30:32.479339+00:00 | 2026-01-19 11:33:17.704037+00:00
  Session 6: 2026-01-19 11:36:04.515965+00:00 | 2026-01-19 11:36:57.219197+00:00
  Session 7: 2026-01-

In [236]:
import pandas as pd

# =========================
# REQUIRED INPUTS
# df: columns ["timestamp_client", "revelations_clean", "mac_list"]
# beacon_macs: set of iBeacon MACs (strings), ideally uppercase
# GAP: pd.Timedelta (e.g., pd.Timedelta(seconds=120))
# =========================

# --- Ensure GAP is a Timedelta (prevents "TypeError: '>' not supported..." etc.) ---
if not isinstance(GAP, pd.Timedelta):
    GAP = pd.to_timedelta(GAP)

# --- Copy + parse timestamps (UTC) ---
df = df.copy()
df["ts_utc"] = pd.to_datetime(df["timestamp_client"], utc=True, errors="coerce")

# --- Keep only usable rows ---
df = df[df["ts_utc"].notna() & df["revelations_clean"].notna()].copy()

# --- Normalize beacon_macs to uppercase strings (avoids mismatch) ---
beacon_macs = {str(m).upper() for m in beacon_macs if pd.notna(m)}

# --- Normalize mac_list into a real list-of-macs for every row ---
def _normalize_mac_list(x):
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return []
    if isinstance(x, list):
        return x
    # if it's a string, try splitting on common separators
    if isinstance(x, str):
        s = x.strip()
        if not s:
            return []
        # handles "['AA', 'BB']" poorly; keep simple
        for sep in ["|", ";", ",", " "]:
            if sep in s:
                parts = [p for p in (pp.strip() for pp in s.split(sep)) if p]
                if len(parts) > 1:
                    return parts
        return [s]
    # fallback: wrap single value
    return [x]

df["mac_list_norm"] = df["mac_list"].apply(_normalize_mac_list)

# --- Explode: each observation keeps the revelations_clean from that MQTT row ---
obs_detail = (
    df[["ts_utc", "revelations_clean", "mac_list_norm"]]
      .explode("mac_list_norm")
      .rename(columns={"mac_list_norm": "mac"})
)

# --- Normalize MAC + filter to iBeacon MACs only ---
obs_detail["mac"] = obs_detail["mac"].astype(str).str.upper().str.strip()
obs_detail = obs_detail[obs_detail["mac"].isin(beacon_macs)].copy()

# --- Drop duplicates + sort ---
obs_detail = (
    obs_detail.drop_duplicates(subset=["mac", "ts_utc"])
              .sort_values(["mac", "ts_utc"])
              .reset_index(drop=True)
)

# --- Assign session_id per MAC using GAP rule ---
def _assign_sessions(group: pd.DataFrame) -> pd.DataFrame:
    g = group.sort_values("ts_utc").copy()
    gaps = g["ts_utc"].diff()
    g["session_id"] = (gaps.isna() | (gaps > GAP)).cumsum().astype(int)  # 1..N
    return g

df_beacon_session_obs = (
    obs_detail.groupby("mac", group_keys=False, sort=False)
              .apply(_assign_sessions)
              .reset_index(drop=True)
)

# --- Keep only lines that mention this MAC (case-insensitive) ---
def _keep_mac_lines(block, mac: str) -> str:
    if not isinstance(block, str):
        return ""
    mac_u = str(mac).upper()
    out = []
    for ln in block.splitlines():
        if mac_u in ln.upper():
            out.append(ln)
    return "\n".join(out)

df_beacon_session_obs["revelations_clean_mac_only"] = df_beacon_session_obs.apply(
    lambda r: _keep_mac_lines(r["revelations_clean"], r["mac"]),
    axis=1
)

# --- Session summary (start/end/count) ---
df_beacon_session_summary = (
    df_beacon_session_obs.groupby(["mac", "session_id"], as_index=False)
    .agg(
        session_start=("ts_utc", "min"),
        session_end=("ts_utc", "max"),
        num_observations=("ts_utc", "count"),
    )
    .sort_values(["mac", "session_id"])
    .reset_index(drop=True)
)

# --- PRINT: per MAC -> per session -> each timestamp + payload text ---
TEXT_FIELD = "revelations_clean_mac_only"

for mac, gmac in df_beacon_session_obs.groupby("mac", sort=False):
    total_sessions = int(gmac["session_id"].max()) if not gmac.empty else 0
    print(f"\nMAC: {mac} | total_sessions: {total_sessions}")

    for sid, gs in gmac.groupby("session_id", sort=True):
        s_start = gs["ts_utc"].min()
        s_end   = gs["ts_utc"].max()
        print(f"  Session {int(sid)}: {s_start} -> {s_end} | observations: {len(gs)}")

        for _, r in gs.iterrows():
            print(f"    - {r['ts_utc']}")
            txt = r[TEXT_FIELD]
            print(txt if isinstance(txt, str) else "")
            print("")


MAC: BC:57:29:02:89:1F | total_sessions: 5
  Session 1: 2026-01-19 09:47:48.977898+00:00 -> 2026-01-19 09:59:39.936850+00:00 | observations: 41
    - 2026-01-19 09:47:48.977898+00:00
[NEW] Device BC:57:29:02:89:1F KBPro_366801

    - 2026-01-19 09:47:51.025386+00:00
[CHG] Device BC:57:29:02:89:1F RSSI is nil
[CHG] Device BC:57:29:02:89:1F RSSI: -82

    - 2026-01-19 09:47:52.971264+00:00
[CHG] Device BC:57:29:02:89:1F RSSI is nil

    - 2026-01-19 09:48:15.297433+00:00
[CHG] Device BC:57:29:02:89:1F RSSI: -76

    - 2026-01-19 09:48:17.345927+00:00
[CHG] Device BC:57:29:02:89:1F RSSI is nil

    - 2026-01-19 09:48:31.372389+00:00
[CHG] Device BC:57:29:02:89:1F RSSI: -76

    - 2026-01-19 09:48:33.420424+00:00
[CHG] Device BC:57:29:02:89:1F RSSI is nil

    - 2026-01-19 09:49:10.594238+00:00
[NEW] Device BC:57:29:02:89:1F KBPro_366801

    - 2026-01-19 09:49:35.169608+00:00
[CHG] Device BC:57:29:02:89:1F RSSI is nil
[CHG] Device BC:57:29:02:89:1F RSSI: -78

    - 2026-01-19 09:49:37.15

In [ ]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
import re
import numpy as np

# ============================================================
# BeaconDeepViewer
#   Tabs order (as requested):
#     1) Inventory
#     2) Presence Summary  (NEW)
#     3) Session Inspector
#     4) Track Devices
#
#   - Presence Summary shows ONLY iBeacon devices:
#       mac | first_seen | last_seen | number_of_sessions | total_present_time | total_absent_time
#
# Notes:
#   - +1 hour DISPLAY offset (data unchanged)
#   - Movement detection uses MAD-based robust std + median(|RSSI|)
# ============================================================

TIME_OFFSET = pd.Timedelta(hours=1)  # adds 1 hour when DISPLAYING timestamps

# --- Near/Far thresholds (edit if you want) ---
NEAR_RSSI_THRESHOLD = -70   # rssi >= -70 => NEAR
FAR_RSSI_THRESHOLD  = -80   # rssi <= -80 => FAR
# Between (-80, -70) => MID_RANGE

# --- Movement thresholds (NO uncertainty) ---
# We use robust std = 1.4826 * MAD instead of plain std
STATIC_STD_MAX = 3.0
STATIC_STEP_MAX = 2.5  # median(|RSSI|)

MOVING_STD_MIN = 4.0
MOVING_STEP_MIN = 3.5  # median(|RSSI|)
# If values are between bands, we FORCE a decision using a score.


class BeaconDeepViewer(tk.Tk):
    _RSSI_RE = re.compile(r"RSSI\s*:\s*(-?\d+)", re.IGNORECASE)

    def __init__(self, df_tuples: pd.DataFrame, df_sessions: pd.DataFrame, df_raw: pd.DataFrame):
        super().__init__()
        self.title("iBeacon Session Inspector")
        self.geometry("1250x820")
        self.minsize(1000, 700)
        self.resizable(True, True)

        self.df_tuples = df_tuples.copy()
        self.df_sessions = df_sessions.copy()
        self.df_raw = df_raw.copy()

        self._prepare_inputs()
        self._current_mac = None

        self._build_ui()

    # ---------------------------------------------------------
    # Data preparation
    # ---------------------------------------------------------
    @staticmethod
    def _normalize_mac_list(x):
        """Return list of uppercase MACs from list/tuple/set/string/None."""
        if x is None or (isinstance(x, float) and pd.isna(x)):
            return []
        if isinstance(x, (list, tuple, set)):
            return [str(m).upper().strip() for m in x if pd.notna(m) and str(m).strip()]
        if isinstance(x, str):
            s = x.strip()
            if not s:
                return []
            for sep in ["|", ";", ",", " "]:
                if sep in s:
                    parts = [p.strip() for p in s.split(sep) if p.strip()]
                    if len(parts) > 1:
                        return [p.upper() for p in parts]
            return [s.upper()]
        return [str(x).upper().strip()]

    def _prepare_inputs(self):
        # ---- iBeacon tuples
        if "mac" not in self.df_tuples.columns:
            raise KeyError("df_tuples must contain 'mac'")

        if "beacon_uuid" in self.df_tuples.columns:
            self.uuid_col = "beacon_uuid"
        elif "uuid" in self.df_tuples.columns:
            self.uuid_col = "uuid"
        else:
            raise KeyError("df_tuples must contain 'beacon_uuid' or 'uuid'")

        for c in ("major", "minor"):
            if c not in self.df_tuples.columns:
                raise KeyError(f"df_tuples must contain '{c}'")

        self.df_tuples["mac"] = self.df_tuples["mac"].astype(str).str.upper().str.strip()

        # keep one tuple row per mac
        self.df_tuples = (
            self.df_tuples
            .sort_values(["mac", self.uuid_col, "major", "minor"])
            .groupby("mac", as_index=False)
            .first()
        )

        self._list_macs = sorted(self.df_tuples["mac"].unique().tolist())
        self._list_uuids = sorted(self.df_tuples[self.uuid_col].dropna().astype(str).str.lower().unique().tolist())
        self._list_majors = sorted(self.df_tuples["major"].dropna().unique().tolist())
        self._list_minors = sorted(self.df_tuples["minor"].dropna().unique().tolist())

        # ---- sessions
        required = {"mac", "session_id", "session_start", "session_end"}
        if not required.issubset(self.df_sessions.columns):
            raise KeyError(f"df_sessions must contain {required}")

        self.df_sessions["mac"] = self.df_sessions["mac"].astype(str).str.upper().str.strip()
        self.df_sessions["session_start"] = pd.to_datetime(self.df_sessions["session_start"], utc=True, errors="coerce")
        self.df_sessions["session_end"] = pd.to_datetime(self.df_sessions["session_end"], utc=True, errors="coerce")
        self.df_sessions = self.df_sessions.dropna(subset=["session_start", "session_end"]).copy()

        # ---- raw observations
        if "revelations_clean" not in self.df_raw.columns:
            raise KeyError("df_raw must contain 'revelations_clean'")
        if "mac_list" not in self.df_raw.columns:
            raise KeyError("df_raw must contain 'mac_list'")

        if "ts_utc" not in self.df_raw.columns:
            if "timestamp_client" in self.df_raw.columns:
                self.df_raw["ts_utc"] = pd.to_datetime(self.df_raw["timestamp_client"], utc=True, errors="coerce")
            else:
                raise KeyError("df_raw must contain 'ts_utc' or 'timestamp_client'")

        self.df_raw = self.df_raw.dropna(subset=["ts_utc"]).copy()
        self.df_raw["mac_list"] = self.df_raw["mac_list"].apply(self._normalize_mac_list)

    # ---------------------------------------------------------
    # Parsing helpers
    # ---------------------------------------------------------
    @classmethod
    def _parse_rssi_from_text(cls, text: str):
        """Extract RSSI ints from a block. Ignores 'RSSI is nil' (no match)."""
        if not isinstance(text, str):
            return []
        vals = []
        for m in cls._RSSI_RE.finditer(text):
            try:
                vals.append(int(m.group(1)))
            except Exception:
                pass
        return vals

    # ---------------------------------------------------------
    # Movement detection (NO uncertainty) — Tab Track Devices
    #   Uses MAD instead of plain std
    # ---------------------------------------------------------
    @staticmethod
    def _detect_moving_or_static_mad(rssi_values):
        """
        Forced decision: returns (label, std, median_step, n)

        std = 1.4826 * MAD
          MAD = median(|r - median(r)|)

        median_step = median(|RSSI|)

        Rules:
          STATIC if std <= STATIC_STD_MAX AND median_step <= STATIC_STEP_MAX
          MOVING if std >= MOVING_STD_MIN OR  median_step >= MOVING_STEP_MIN
          else: forced decision via score (no uncertainty)
        """
        rssi_values = [int(v) for v in (rssi_values or []) if v is not None]
        n = len(rssi_values)

        if n < 5:
            return ("STATIC", None, None, n)

        r = np.array(rssi_values, dtype=float)

        med = float(np.median(r))
        mad = float(np.median(np.abs(r - med)))
        std = 1.4826 * mad

        median_step = float(np.median(np.abs(np.diff(r)))) if n >= 2 else 0.0

        if std <= STATIC_STD_MAX and median_step <= STATIC_STEP_MAX:
            return ("STATIC", std, median_step, n)

        if std >= MOVING_STD_MIN or median_step >= MOVING_STEP_MIN:
            return ("MOVING", std, median_step, n)

        # Force decision
        std_mid = (STATIC_STD_MAX + MOVING_STD_MIN) / 2.0   # 3.5
        step_mid = (STATIC_STEP_MAX + MOVING_STEP_MIN) / 2.0  # 3.0
        score = 0.6 * (std - std_mid) + 0.4 * (median_step - step_mid)
        label = "MOVING" if score >= 0 else "STATIC"
        return (label, std, median_step, n)

    # ---------------------------------------------------------
    # Proximity (NEAR/FAR/MID_RANGE)
    # ---------------------------------------------------------
    @staticmethod
    def _proximity_label(rssi: int) -> str:
        try:
            r = int(rssi)
        except Exception:
            return None

        if r >= NEAR_RSSI_THRESHOLD:
            return "NEAR"
        if r <= FAR_RSSI_THRESHOLD:
            return "FAR"
        return "MID_RANGE"

    # ---------------------------------------------------------
    # MAC-scoped event extraction (Session Inspector)
    # ---------------------------------------------------------
    @staticmethod
    def extract_events_for_mac(block: str, mac: str) -> str:
        if not isinstance(block, str):
            return ""

        mac = mac.upper()
        out = []
        capturing = False

        for ln in block.splitlines():
            if ln.lstrip().startswith("["):
                if mac in ln.upper():
                    capturing = True
                    out.append(ln)
                else:
                    capturing = False
            else:
                if capturing:
                    out.append(ln)

        return "\n".join(out).strip()

    # ---------------------------------------------------------
    # Inventory formatting (Tab Inventory)
    # ---------------------------------------------------------
    def _inventory_text(self) -> str:
        lines = []
        lines.append("Beacon Inventory Summary\n")

        lines.append(f"Total unique MACs   : {len(self._list_macs)}")
        lines.append("---- MACs ----")
        lines.extend(f"  {m}" for m in self._list_macs)
        lines.append("")

        lines.append(f"Total unique UUIDs  : {len(self._list_uuids)}")
        lines.append("---- UUIDs ----")
        lines.extend(f"  {u}" for u in self._list_uuids)
        lines.append("")

        lines.append(f"Total unique Majors : {len(self._list_majors)}")
        lines.append("---- Majors ----")
        lines.extend(f"  {m}" for m in self._list_majors)
        lines.append("")

        lines.append(f"Total unique Minors : {len(self._list_minors)}")
        lines.append("---- Minors ----")
        lines.extend(f"  {m}" for m in self._list_minors)

        return "\n".join(lines)

    # ---------------------------------------------------------
    # Presence Summary computation (Tab Presence Summary)
    # ---------------------------------------------------------
    @staticmethod
    def _fmt_tdelta_hms(td) -> str:
        if td is None or pd.isna(td):
            return ""
        try:
            total_seconds = int(td.total_seconds())
        except Exception:
            return ""
        sign = "-" if total_seconds < 0 else ""
        total_seconds = abs(total_seconds)
        h = total_seconds // 3600
        m = (total_seconds % 3600) // 60
        s = total_seconds % 60
        return f"{sign}{h}:{m:02d}:{s:02d}"

    def _compute_presence_summary(self) -> pd.DataFrame:
        """
        For each iBeacon MAC (from df_tuples):
          - first_seen: min(session_start)
          - last_seen : max(session_end)
          - number_of_sessions: count
          - total_present_time: sum(session_end - session_start)
          - total_absent_time: sum(next_start - current_end) for positive gaps
        """
        if self.df_sessions.empty:
            return pd.DataFrame(columns=[
                "mac", "first_seen", "last_seen",
                "number_of_sessions", "total_present_time", "total_absent_time"
            ])

        beacon_macs = set(self.df_tuples["mac"].astype(str).str.upper().str.strip().tolist())
        ds = self.df_sessions.copy()
        ds["mac"] = ds["mac"].astype(str).str.upper().str.strip()
        ds = ds[ds["mac"].isin(beacon_macs)].copy()

        out_rows = []
        for mac, g in ds.groupby("mac", sort=False):
            g = g.sort_values("session_start").reset_index(drop=True)

            first_seen = g["session_start"].min()
            last_seen = g["session_end"].max()
            num_sessions = int(g.shape[0])

            present = (g["session_end"] - g["session_start"]).sum()

            gaps = g["session_start"].shift(-1) - g["session_end"]
            gaps = gaps.dropna()
            gaps = gaps[gaps > pd.Timedelta(0)]
            absent = gaps.sum() if not gaps.empty else pd.Timedelta(0)

            out_rows.append({
                "mac": mac,
                "first_seen": first_seen,
                "last_seen": last_seen,
                "number_of_sessions": num_sessions,
                "total_present_time": present,
                "total_absent_time": absent
            })

        dfp = pd.DataFrame(out_rows)
        if dfp.empty:
            return dfp

        dfp = dfp.sort_values("mac").reset_index(drop=True)
        return dfp

    # ---------------------------------------------------------
    # UI: Tabs (order requested)
    # ---------------------------------------------------------
    def _build_ui(self):
        nb = ttk.Notebook(self)
        nb.pack(fill="both", expand=True)

        self.tab_inventory = ttk.Frame(nb)     # 1
        self.tab_presence = ttk.Frame(nb)      # 2 (NEW)
        self.tab_inspect = ttk.Frame(nb)       # 3
        self.tab_track = ttk.Frame(nb)         # 4

        nb.add(self.tab_inventory, text="Inventory")
        nb.add(self.tab_presence, text="Presence Summary")
        nb.add(self.tab_inspect, text="Session Inspector")
        nb.add(self.tab_track, text="Track Devices")

        self._build_tab_inventory(self.tab_inventory)
        self._build_tab_presence_summary(self.tab_presence)
        self._build_tab_session_inspector(self.tab_inspect)
        self._build_tab_track_devices(self.tab_track)

    # ---------------------------------------------------------
    # Tab 1: Inventory
    # ---------------------------------------------------------
    def _build_tab_inventory(self, parent):
        paned = ttk.Panedwindow(parent, orient="vertical")
        paned.pack(fill="both", expand=True, padx=10, pady=10)

        top = ttk.Frame(paned)
        bottom = ttk.Frame(paned)

        paned.add(top, weight=2)
        paned.add(bottom, weight=3)

        txt_box = ttk.LabelFrame(top, text="Beacon Inventory Summary")
        txt_box.pack(fill="both", expand=True)

        self.inventory_text = tk.Text(txt_box, wrap="word")
        self.inventory_text.pack(fill="both", expand=True)
        self.inventory_text.insert("1.0", self._inventory_text())
        self.inventory_text.configure(state="disabled")

        tbl_box = ttk.LabelFrame(bottom, text="iBeacon Tuples (MAC → UUID, Major, Minor)")
        tbl_box.pack(fill="both", expand=True)

        self.inv_table = ttk.Treeview(
            tbl_box,
            columns=("mac", "uuid", "major", "minor"),
            show="headings",
            height=16
        )
        self.inv_table.heading("mac", text="mac")
        self.inv_table.heading("uuid", text="uuid")
        self.inv_table.heading("major", text="major")
        self.inv_table.heading("minor", text="minor")

        self.inv_table.column("mac", anchor="w", width=220, stretch=False)
        self.inv_table.column("uuid", anchor="w", width=520, stretch=True)
        self.inv_table.column("major", anchor="center", width=90, stretch=False)
        self.inv_table.column("minor", anchor="center", width=90, stretch=False)

        yscroll = ttk.Scrollbar(tbl_box, orient="vertical", command=self.inv_table.yview)
        self.inv_table.configure(yscrollcommand=yscroll.set)

        self.inv_table.pack(side="left", fill="both", expand=True)
        yscroll.pack(side="right", fill="y")

        df_inv = self.df_tuples.copy()
        df_inv = df_inv.sort_values(["mac", self.uuid_col, "major", "minor"]).reset_index(drop=True)

        for _, r in df_inv.iterrows():
            self.inv_table.insert(
                "", "end",
                values=(str(r["mac"]), str(r[self.uuid_col]), str(r["major"]), str(r["minor"]))
            )

    # ---------------------------------------------------------
    # Tab 2: Presence Summary (NEW)
    # ---------------------------------------------------------
    def _build_tab_presence_summary(self, parent):
        box = ttk.LabelFrame(parent, text="iBeacon Presence Summary (from sessions)")
        box.pack(fill="both", expand=True, padx=10, pady=10)

        self.pres_table = ttk.Treeview(
            box,
            columns=("mac", "first", "last", "nsess", "present", "absent"),
            show="headings",
            height=20
        )
        self.pres_table.heading("mac", text="mac")
        self.pres_table.heading("first", text="first_seen")
        self.pres_table.heading("last", text="last_seen")
        self.pres_table.heading("nsess", text="sessions")
        self.pres_table.heading("present", text="total_present_time")
        self.pres_table.heading("absent", text="total_absent_time")

        self.pres_table.column("mac", anchor="w", width=220, stretch=False)
        self.pres_table.column("first", anchor="w", width=260, stretch=True)
        self.pres_table.column("last", anchor="w", width=260, stretch=True)
        self.pres_table.column("nsess", anchor="center", width=90, stretch=False)
        self.pres_table.column("present", anchor="center", width=160, stretch=False)
        self.pres_table.column("absent", anchor="center", width=160, stretch=False)

        yscroll = ttk.Scrollbar(box, orient="vertical", command=self.pres_table.yview)
        self.pres_table.configure(yscrollcommand=yscroll.set)

        self.pres_table.pack(side="left", fill="both", expand=True)
        yscroll.pack(side="right", fill="y")

        # Fill table
        dfp = self._compute_presence_summary()
        for _, r in dfp.iterrows():
            first_disp = (r["first_seen"] + TIME_OFFSET).isoformat() if pd.notna(r["first_seen"]) else ""
            last_disp = (r["last_seen"] + TIME_OFFSET).isoformat() if pd.notna(r["last_seen"]) else ""
            self.pres_table.insert(
                "", "end",
                values=(
                    r["mac"],
                    first_disp,
                    last_disp,
                    int(r["number_of_sessions"]),
                    self._fmt_tdelta_hms(r["total_present_time"]),
                    self._fmt_tdelta_hms(r["total_absent_time"]),
                )
            )

    # ---------------------------------------------------------
    # Tab 3: Session Inspector (same behavior)
    # ---------------------------------------------------------
    def _build_tab_session_inspector(self, parent):
        top = ttk.Frame(parent)
        top.pack(fill="x", padx=10, pady=6)

        ttk.Label(top, text="Beacon MAC:").pack(side="left")
        self.mac_var = tk.StringVar()

        self.mac_combo = ttk.Combobox(
            top, values=self._list_macs, textvariable=self.mac_var, state="readonly", width=30
        )
        self.mac_combo.pack(side="left", padx=6)
        self.mac_combo.bind("<<ComboboxSelected>>", self.on_mac_selected)

        self.meta_label = ttk.Label(top, text="", font=("Courier", 10))
        self.meta_label.pack(side="left", padx=18)

        mid = ttk.LabelFrame(parent, text="Sessions")
        mid.pack(fill="both", expand=True, padx=10, pady=6)

        self.session_table = ttk.Treeview(
            mid,
            columns=("sid", "start", "end", "duration", "next_gap"),
            show="headings",
            height=9
        )

        col_defs = {
            "sid": ("sid", 140),
            "start": ("start", 300),
            "end": ("end", 300),
            "duration": ("duration", 120),
            "next_gap": ("next_gap", 120),
        }

        for c, (label, w) in col_defs.items():
            self.session_table.heading(c, text=label)
            self.session_table.column(c, anchor="center", width=w, stretch=True)

        yscroll = ttk.Scrollbar(mid, orient="vertical", command=self.session_table.yview)
        self.session_table.configure(yscrollcommand=yscroll.set)

        self.session_table.pack(side="left", fill="both", expand=True)
        yscroll.pack(side="right", fill="y")

        self.session_table.bind("<<TreeviewSelect>>", self.on_session_selected)

        bottom = ttk.LabelFrame(parent, text="Session (timestamp → events)")
        bottom.pack(fill="both", expand=True, padx=10, pady=6)

        self.evidence = tk.Text(bottom, wrap="word")
        self.evidence.pack(fill="both", expand=True)

    # ---------------------------------------------------------
    # Tab 4: Track Devices (same behavior)
    # ---------------------------------------------------------
    def _build_tab_track_devices(self, parent):
        top = ttk.Frame(parent)
        top.pack(fill="x", padx=10, pady=6)

        ttk.Label(top, text="Beacon MAC:").pack(side="left")
        self.track_mac_var = tk.StringVar()

        self.track_mac_combo = ttk.Combobox(
            top, values=self._list_macs, textvariable=self.track_mac_var, state="readonly", width=30
        )
        self.track_mac_combo.pack(side="left", padx=6)
        self.track_mac_combo.bind("<<ComboboxSelected>>", self.on_track_mac_selected)

        self.track_meta_label = ttk.Label(top, text="", font=("Courier", 10))
        self.track_meta_label.pack(side="left", padx=18)

        paned = ttk.Panedwindow(parent, orient="horizontal")
        paned.pack(fill="both", expand=True, padx=10, pady=6)

        left = ttk.Frame(paned)
        right = ttk.Frame(paned)
        paned.add(left, weight=4)
        paned.add(right, weight=2)

        # LEFT table
        left_box = ttk.LabelFrame(left, text="Timestamp → RSSI → Proximity")
        left_box.pack(fill="both", expand=True)

        self.track_table = ttk.Treeview(
            left_box,
            columns=("ts", "rssi", "prox"),
            show="headings",
            height=18
        )
        self.track_table.heading("ts", text="timestamp")
        self.track_table.heading("rssi", text="rssi")
        self.track_table.heading("prox", text="NEAR/FAR/MID_RANGE")

        self.track_table.column("ts", anchor="w", width=520, stretch=True)
        self.track_table.column("rssi", anchor="center", width=90, stretch=False)
        self.track_table.column("prox", anchor="center", width=160, stretch=False)

        yscroll = ttk.Scrollbar(left_box, orient="vertical", command=self.track_table.yview)
        self.track_table.configure(yscrollcommand=yscroll.set)

        self.track_table.pack(side="left", fill="both", expand=True)
        yscroll.pack(side="right", fill="y")

        # RIGHT summary
        right_box = ttk.LabelFrame(right, text="Decision Summary")
        right_box.pack(fill="both", expand=True)

        self.track_summary_text = tk.Text(right_box, wrap="word")
        self.track_summary_text.pack(fill="both", expand=True)
        self.track_summary_text.insert("1.0", "Select a MAC to compute decisions.\n")
        self.track_summary_text.configure(state="disabled")

    # ---------------------------------------------------------
    # Handlers (Session Inspector)
    # ---------------------------------------------------------
    @staticmethod
    def _fmt_tdelta(td) -> str:
        if td is None or pd.isna(td):
            return ""
        try:
            total_seconds = int(td.total_seconds())
        except Exception:
            return ""
        sign = "-" if total_seconds < 0 else ""
        total_seconds = abs(total_seconds)
        h = total_seconds // 3600
        m = (total_seconds % 3600) // 60
        s = total_seconds % 60
        return f"{sign}{h}:{m:02d}:{s:02d}"

    def on_mac_selected(self, event=None):
        mac = self.mac_var.get().upper().strip()
        self._current_mac = mac

        row = self.df_tuples[self.df_tuples["mac"] == mac]
        if not row.empty:
            r = row.iloc[0]
            self.meta_label.config(text=f"UUID={r[self.uuid_col]} | Major={r['major']} | Minor={r['minor']}")
        else:
            self.meta_label.config(text="")

        self.session_table.delete(*self.session_table.get_children())

        dfm = self.df_sessions[self.df_sessions["mac"] == mac].copy()
        if dfm.empty:
            self.evidence.delete("1.0", "end")
            return

        dfm = dfm.sort_values("session_start").reset_index(drop=True)
        dfm["duration"] = dfm["session_end"] - dfm["session_start"]
        dfm["next_gap"] = dfm["session_start"].shift(-1) - dfm["session_end"]

        for _, r in dfm.iterrows():
            self.session_table.insert("", "end", values=(
                int(r["session_id"]) if pd.notna(r["session_id"]) else "",
                (r["session_start"] + TIME_OFFSET).isoformat(),
                (r["session_end"] + TIME_OFFSET).isoformat(),
                self._fmt_tdelta(r["duration"]),
                self._fmt_tdelta(r["next_gap"]),
            ))

        self.evidence.delete("1.0", "end")

    def on_session_selected(self, event=None):
        sel = self.session_table.selection()
        if not sel:
            return

        sid, start, end, *_ = self.session_table.item(sel[0])["values"]

        start = pd.to_datetime(start, utc=True, errors="coerce") - TIME_OFFSET
        end = pd.to_datetime(end, utc=True, errors="coerce") - TIME_OFFSET
        if pd.isna(start) or pd.isna(end):
            return

        mac = self._current_mac
        self.evidence.delete("1.0", "end")

        mask = (
            self.df_raw["ts_utc"].between(start, end, inclusive="both") &
            self.df_raw["mac_list"].apply(lambda xs: mac in xs)
        )

        for _, r in self.df_raw.loc[mask].sort_values("ts_utc").iterrows():
            ev = self.extract_events_for_mac(r["revelations_clean"], mac)
            if ev:
                self.evidence.insert("end", f"- {(r['ts_utc'] + TIME_OFFSET).isoformat()}\n{ev}\n\n")

    # ---------------------------------------------------------
    # Handler (Track Devices)
    # ---------------------------------------------------------
    def on_track_mac_selected(self, event=None):
        mac = self.track_mac_var.get().upper().strip()

        row = self.df_tuples[self.df_tuples["mac"] == mac]
        if not row.empty:
            r = row.iloc[0]
            self.track_meta_label.config(text=f"UUID={r[self.uuid_col]} | Major={r['major']} | Minor={r['minor']}")
        else:
            self.track_meta_label.config(text="")

        # Clear table + summary
        self.track_table.delete(*self.track_table.get_children())
        self.track_summary_text.configure(state="normal")
        self.track_summary_text.delete("1.0", "end")
        self.track_summary_text.configure(state="disabled")

        mask = self.df_raw["mac_list"].apply(lambda xs: mac in xs)
        rows = self.df_raw.loc[mask].sort_values("ts_utc")

        samples = []   # (ts_utc, rssi, prox_label)
        rssi_only = []
        prox_counts = {"NEAR": 0, "FAR": 0, "MID_RANGE": 0}

        for _, rr in rows.iterrows():
            ev = self.extract_events_for_mac(rr["revelations_clean"], mac)
            text_to_parse = ev if ev else rr["revelations_clean"]

            vals = self._parse_rssi_from_text(text_to_parse)
            for v in vals:
                prox = self._proximity_label(v)
                if prox is None:
                    continue

                samples.append((rr["ts_utc"], v, prox))
                rssi_only.append(v)
                prox_counts[prox] += 1

        for ts, v, prox in samples:
            self.track_table.insert("", "end", values=((ts + TIME_OFFSET).isoformat(), int(v), prox))

        total_prox = sum(prox_counts.values())
        prox_major = max(prox_counts.items(), key=lambda kv: kv[1])[0] if total_prox else "MID_RANGE"

        motion_label, std, median_step, n = self._detect_moving_or_static_mad(rssi_only)

        lines = []
        lines.append(f"MAC: {mac}")
        lines.append("")
        lines.append("1) Proximity Status (NEAR / FAR / MID_RANGE)")
        lines.append("   Thresholds:")
        lines.append(f"     NEAR      if rssi >= {NEAR_RSSI_THRESHOLD}")
        lines.append(f"     FAR       if rssi <= {FAR_RSSI_THRESHOLD}")
        lines.append(f"     MID_RANGE if {FAR_RSSI_THRESHOLD} < rssi < {NEAR_RSSI_THRESHOLD}")
        lines.append("")
        lines.append(f"   Total RSSI samples: {total_prox}")
        lines.append("   Counts:")
        lines.append(f"     NEAR     : {prox_counts['NEAR']}")
        lines.append(f"     MID_RANGE: {prox_counts['MID_RANGE']}")
        lines.append(f"     FAR      : {prox_counts['FAR']}")
        lines.append("")
        lines.append("")
        lines.append("2) Behavior Decision (MOVING / STATIC) [MAD-based]")
        lines.append("   Metrics:")
        lines.append("     std = 1.4826 * MAD")
        lines.append("     MAD = median(|RSSI - median(RSSI)|)")
        lines.append("     step = median(|RSSI|)")
        lines.append("")
        lines.append("   Thresholds:")
        lines.append(f"     STATIC if std <= {STATIC_STD_MAX} AND step <= {STATIC_STEP_MAX}")
        lines.append(f"     MOVING if std >= {MOVING_STD_MIN} OR  step >= {MOVING_STEP_MIN}")
        lines.append("     Otherwise: forced decision using std+step score")
        lines.append("")

        if std is None or median_step is None:
            lines.append(f"   Decision: {motion_label} | samples={n} (not enough data for stable stats)")
        else:
            lines.append(f"   std        : {std:.2f}")
            lines.append(f"   step       : {median_step:.2f}")
            lines.append(f"   Decision   : {motion_label}")

        lines.append("")
        lines.append("Combined:")
        lines.append(f"  Proximity majority = {prox_major}")
        lines.append(f"  Movement behavior  = {motion_label}")

        self.track_summary_text.configure(state="normal")
        self.track_summary_text.insert("1.0", "\n".join(lines))
        self.track_summary_text.configure(state="disabled")


# ============================================================
# RUN
# ============================================================
app = BeaconDeepViewer(
    mac_beacon_agg,
    df_beacon_session_detail,
    df
)
app.mainloop()